In [ ]:
import affine, concurrent, cStringIO, glob, IPython, json, os, PIL, sys, thread, time, traceback

In [ ]:
class SimpleProcessPoolExecutor(concurrent.futures.ProcessPoolExecutor):
    def __init__(self, max_workers):
        super(SimpleProcessPoolExecutor, self).__init__(max_workers=max_workers)
        self.futures = []
        
    def submit(self, fn, *args, **kwargs):
        future = super(SimpleProcessPoolExecutor, self).submit(fn, *args, **kwargs)
        self.futures.append(future)
        return future
    
    def get_futures(self):
        return self.futures

    def shutdown(self):
        exception_count = 0
        results = []
        for completed in concurrent.futures.as_completed(self.futures):
            try:
                results.append(completed.result())
            except Exception as e:
                exception_count += 1
                sys.stderr.write(
                    'Exception caught in SimpleProcessPoolExecutor.shutdown.  Continuing until all are finished.\n' +
                    'Exception follows:\n' +
                    traceback.format_exc())
        super(SimpleProcessPoolExecutor, self).shutdown()
        if exception_count:
            raise Exception('SimpleProcessPoolExecutor failed: %d of %d raised exception' % (exception_count, len(self.futures)))
        print 'SimpleProcessPoolExecutor succeeded: all %d jobs completed' % (len(self.futures))
        return results

class Stopwatch:
    def __init__(self, name):
        self.name = name
    def __enter__(self):
        self.start = time.time()
    def __exit__(self, type, value, traceback):
        sys.stdout.write('%s took %.1f seconds\n' % (self.name, time.time() - self.start))


In [ ]:
def select_into(tname):
    c = """
    SELECT 
      %s_gid, 
      sum(st_area(geom::geography))      
    INTO TABLE %s_areas
    FROM %s_subdivided
    GROUP BY %s_gid 
    ORDER BY %s_gid
    ;
    """
    c = c % (tname, tname, tname, tname, tname)
    cmd = 'psql -d biodiversity -c "%s"' % c 
    !$cmd            

def drop_table(tname):
    c = """
    DROP TABLE %s_areas
    ;
    """
    c = c % (tname)
    cmd = 'psql -d biodiversity -c "%s"' % c 
    !$cmd            

def create_index(tname):
    c = "CREATE UNIQUE INDEX IF NOT EXISTS %s_gid_idx ON %s_areas (%s_gid);" % (tname, tname, tname)
    cmd = 'psql -d biodiversity -c "%s"' % c 
    !$cmd    

def vacuum_table(tname):
    c = "VACUUM ANALYZE %s_areas;" % tname
    cmd = 'psql -d biodiversity -c "%s"' % c 
    !$cmd   
    

In [ ]:
tnames = []
for f in glob.glob("../../data/redlist/*/*.shp"):
    name = os.path.basename(f).split(".")[0]
    tnames.append(name.lower())
tnames.sort()

In [ ]:
len(tnames)

In [ ]:
exe = SimpleProcessPoolExecutor(max_workers=16)
for tname in tnames:
    exe.submit(select_into, tname)
exe.shutdown()
None

In [ ]:
for tname in tnames:
    create_index(tname)


In [ ]:
for tname in tnames:
    vacuum_table(tname)


In [ ]:
#for tname in tnames:
#    drop_table(tname)
